### Basic implementation of an agent using Litemind

## make sure text completion works with base OpenAI wrapper (not using litemind)

In [1]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Endless calls unwind,  \nFunctions within functions,  \nLooping back to start.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [2]:
from litemind.apis.combined_api import CombinedApi
from litemind.agent.tools.toolset import ToolSet
from litemind.apis.providers.openai.openai_api import OpenAIApi
from litemind.agent.agent import Agent


import pymmcore_plus
import numpy as np
import matplotlib.pyplot as plt

# install local openai

import os
import openai
import google


├ OpenAI API key is valid.


Traceback (most recent call last):
  File "c:\Users\dario\anaconda3\envs\masterThesis\lib\site-packages\litemind\apis\providers\ollama\ollama_api.py", line 62, in __init__
    self._model_list = _get_ollama_models_list(self.client)
  File "c:\Users\dario\anaconda3\envs\masterThesis\lib\site-packages\litemind\apis\providers\ollama\utils\list_models.py", line 7, in _get_ollama_models_list
    model_list = list(client.list().models)
  File "c:\Users\dario\anaconda3\envs\masterThesis\lib\site-packages\ollama\_client.py", line 567, in list
    return self._request(
  File "c:\Users\dario\anaconda3\envs\masterThesis\lib\site-packages\ollama\_client.py", line 178, in _request
    return cls(**self._request_raw(*args, **kwargs).json())
  File "c:\Users\dario\anaconda3\envs\masterThesis\lib\site-packages\ollama\_client.py", line 124, in _request_raw
    raise ConnectionError(CONNECTION_ERROR_MESSAGE) from None
ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded,

├ API OllamaApi could not be instantiated: Error initializing Ollama client: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download. Removing it from the list.
├ API GeminiApi could not be instantiated: A valid GOOGLE_API_KEY is required for GeminiApi. Set GOOGLE_API_KEY in the environment or pass api_key explicitly.. Removing it from the list.


In [3]:
def snap(core: pymmcore_plus.CMMCorePlus.instance()):

    # snap and image
    core.snapImage()

    return core.getImage()

In [8]:
# define a tool to snap an image
def get_image(request: str) -> np.array:
    """ The microscope return the image as an array """
    from pymmcore_plus import CMMCorePlus

    core = CMMCorePlus()
    # load configuration
    core.loadSystemConfiguration()
    
    # return the array with the snapped image
    return snap(core)

In [9]:
# create a toolset and add the function tool
toolset = ToolSet()
toolset.add_function_tool(get_image)

api = OpenAIApi()
agent = Agent(api=api, toolset= toolset,model_name="gpt-4o-mini")


# Add a system message to define the agent's behavior
agent.append_system_message("You are a helpful scientist assistant.")

# Run the agent with a user query
response = agent("Take an image of the sample.")

# Print the response
print(response[-1])


├╗ Calling agent: 'Agent'
│├╗ API and model:
││├ API: OpenAIApi
││├ Model: gpt-4o-mini
││┴« 5.35 milliseconds
││
│├╗ Available tools
││├ get_image(request: string) % The microscope return the image as an array 
││┴« 3.00 milliseconds
││
│├╗ Last message in conversation:
││├ *user*:
│││ Take an image of the sample.
││┴« 5.00 milliseconds
││
│├╗ Executing tool 'get_image'
││├ Arguments: (), {'request': 'Take an image of the sample.'}
││├ Result: [[3276 3339 3402 ... 3087 3150 3213]
│││  [3284 3347 3410 ... 3095 3158 3221]
│││  [3292 3355 3418 ... 3103 3166 3229]
│││  ...
│││  [5846 5846 5844 ... 5837 5842 5845]
│││  [5846 5846 5844 ... 5838 5842 5845]
│││  [5846 5846 5843 ... 5839 5843 5845]]
││┴« 125.25 milliseconds
││
│├╗ Reponse:
││├ *assistant*:
│││ TOOL: get_image(request=Take an image of the sample.)
││├ *user*:
│││ TOOL: get_image(request=Take an image of the sample.)="[[3276 3339 3402 ... 3087 3150 3213]\n [3284 3347 3410 ... 3095 3158 3221]\n [3292 3355 3418 ... 3103 3166 3229]\

In [10]:
from litemind.agent.messages.message import Message

# Create messages
system_message = Message(role="system", text="You are a helpful assistant.")
user_message = Message(role="user", text="Take an image of the sample.")

# Generate a response
response = api.generate_text(messages=[system_message, user_message],model_name="gpt-4o-mini")

# Print the response
print(response[0])
# Expected output: "The capital of France is Paris."

*assistant*:
I'm sorry, but I can't take or display images. However, I can help you with descriptions, explanations, or any information you need. Let me know how I can assist you!

